# Performing a Threat Hunt using Python

In [97]:
import paramiko, sys, os, re
from getpass import getpass

In [98]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information to log into the remote system
host = "192.168.6.71"
port = 2222
username = "cole.davisbrand"
password = thePass

# Connect to remote system
try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication failed")

In [99]:
# Upload the Kraken binary
sftp = ssh.open_sftp()
sftp.put('kraken', "/home/cole.davisbrand/kraken")

<SFTPAttributes: [ size=9962192 uid=1286604553 gid=1286603265 mode=0o100755 atime=1667269896 mtime=1667270032 ]>

In [100]:
# Give Kraken executable perms
ssh.exec_command("chmod +x /home/cole.davisbrand/kraken")

# Create Kraken command
krakenCommand = "sudo -S /home/cole.davisbrand/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin"

# Get output from the Kraken command
stdin, stdout, stderr, = ssh.exec_command(krakenCommand)

# Must use password when running commmand with sudo privs
stdin.write(password + "\n")

# print("Kraken has finished executing")

# Create file of PIDs from Kraken output, credit Paul Gleason for the help on this!
for line in stdout.readlines():
    if re.findall(r'.*pid.*', line):
        with open('pid.txt', 'w') as f:
            f.write(line.split(" ")[-2].split("=")[-1] + ' ')


In [101]:
# Run "lsof -p" against PIDs found in Kraken output

lines = []

with open('pid.txt', 'r') as f:
    for line in f:
        lines.append(line)

for item in lines:
    splitLines = item.split(" ")

for t in splitLines:

    command = "sudo -S lsof -p " + t

    stdin, stdout, stderr = ssh.exec_command(command)
    stdin.write(thePass + "\n")
    stdin.flush()

    # Write output to a file for better visability
    for s in stdout.readlines():
        with open('lsofOutput.txt', 'a') as f:
            f.write(s)

# From here I found that the user p.dalton seemed suspicious based out the lsof -p output. Looking at the lsofOutput.txt
# file it was clear that this neeeded to be looked into further

In [96]:
# Download files from suspicious user

